# Shanky Diagram OS-CVD

In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Link Score

In [115]:
link = pd.read_csv("result/link-os-cvd.csv")
cvdTree = pd.read_csv("input/CVD-mesh-nodes.csv")
osTree = pd.read_csv("input/OS-mesh-nodes.csv")

In [116]:
link.head(1)

,Unnamed: 0,CVD,OS,Score
0,10807,myocardial infarction,nitric oxide,73225.0


In [117]:
cvdTree.head(1)

,Unnamed: 0,Name,ID,CVD
0,0,cardiomyopathies,D009202,CM


In [118]:
osTree.head(1)

,Unnamed: 0,Name,Event,Function,ID
0,0,superoxides,Initiation of Oxidative Stress,Reactive Oxygen Species (ROS),D013481


In [119]:
entity = 'myocardial infarction'
list(cvdTree.query("Name ==@entity")['CVD'])[0]

'IHD'

In [120]:
link['cvdCat'] = link['CVD'].apply(lambda x :list(cvdTree.query("Name ==@x")['CVD'])[0])

In [121]:
link['osEvent'] = link['OS'].apply(lambda x :list(osTree.query("Name ==@x")['Event'])[0] )

In [122]:
link['osFunction'] = link['OS'].apply(lambda x :list(osTree.query("Name ==@x")['Function'])[0] )

In [123]:
link.head(2)

,Unnamed: 0,CVD,OS,Score,cvdCat,osEvent,osFunction
0,10807,myocardial infarction,nitric oxide,73225.0,IHD,Initiation of Oxidative Stress,Reactive Nitrogen Species (RNS)
1,10833,myocardial infarction,isoenzymes,72280.0,IHD,Modulation of Oxidative Stress,Antioxidant Enzymes


#### Add more Link scores

In [126]:
def scorer(df, scol, tcol, source, target):
    df_source = df[df[scol] == source]
    df_target = df_source[df_source[tcol] == target]
    score =  df_target['Score'].sum()
    return score

In [127]:
scorer(link,"CVD","cvdCat", "heart failure", "CM")

4092116.0

In [128]:
scorer(link, "cvdCat","osEvent", "CM", "Initiation of Oxidative Stress")

1308868.0

In [149]:
scorer(link,"osEvent", "osFunction", "Initiation of Oxidative Stress", "Reactive Nitrogen Species (RNS)")

2327774.0

In [130]:
scorer(link,"osFunction", "OS", "Reactive Nitrogen Species (RNS)", "nitric oxide")

1006211.0

### CVD Tree

In [177]:
cvdTree.head(5)

,Unnamed: 0,Name,ID,CVD
0,0,cardiomyopathies,D009202,CM
1,1,arrhythmogenic right ventricular dysplasia,D019571,CM
2,2,"cardiomyopathy, alcoholic",D002310,CM
3,3,"cardiomyopathy, dilated",D002311,CM
4,4,"cardiomyopathy, hypertrophic",D002312,CM


In [178]:
list(cvdTree['CVD'].unique())

['CM', 'ARR', 'CHD', 'VD', 'IHD', 'CCS', 'VOO', 'OHD']

In [179]:
label = list(cvdTree['Name']) + list(cvdTree['CVD'].unique())
labelindex = [label.index(x) for x in label]
label2index = {}
for item, index in zip(label,labelindex):
    label2index.update({item:index})

In [180]:
source, target, value = [],[],[]
for mesh, cat in zip(cvdTree['Name'], cvdTree['CVD']):
    source.append(label2index[mesh])
    target.append(label2index[cat])
    value.append(scorer(link,"CVD","cvdCat", mesh, cat))

### OS tree

In [181]:
osTree.head(5)

,Unnamed: 0,Name,Event,Function,ID
0,0,superoxides,Initiation of Oxidative Stress,Reactive Oxygen Species (ROS),D013481
1,1,hydrogen peroxide,Initiation of Oxidative Stress,Reactive Oxygen Species (ROS),D006861
2,2,hydroxyl radical,Initiation of Oxidative Stress,Reactive Oxygen Species (ROS),D017665
3,3,singlet oxygen,Initiation of Oxidative Stress,Reactive Oxygen Species (ROS),D026082
4,4,lipid peroxides,Initiation of Oxidative Stress,Reactive Oxygen Species (ROS),D008054


#### CVD category to OS Event

In [182]:
event = ['IOS','MOS','OOS']

label = label + event 
labelindex = [label.index(x) for x in label]
label2index = {}
for item, index in zip(label,labelindex):
    label2index.update({item:index})

In [183]:
oscat = {'IOS':'Initiation of Oxidative Stress',\
         'MOS':'Modulation of Oxidative Stress',\
         'OOS':'Outcomes of Oxidative Stress'}

In [184]:
cvdcat = list(cvdTree['CVD'].unique())
for cvd in cvdcat:
    for osevent in event:
        source.append(label2index[cvd])
        target.append(label2index[osevent])
        value.append(scorer(link, "cvdCat","osEvent", cvd, osevt[osevent]))
        

#### OS Event to Functions

In [185]:
list(osTree["Event"].unique())

['Initiation of Oxidative Stress',
 'Modulation of Oxidative Stress',
 'Outcomes of Oxidative Stress']

In [186]:
list(osTree["Function"].unique())

['Reactive Oxygen Species (ROS)',
 'Reactive Nitrogen Species (RNS)',
 'Reactive Aldehydes',
 'Redox Metabolites',
 'Antioxidant Enzymes',
 'Redox Regulating Proteins',
 'Antioxidants',
 'Protein Oxidation (O-PTMs)',
 'Lipid Peroxidation Products',
 'Oxidative DNA Damage',
 'Nitrative DNA Damage']

In [187]:
ios_function = list(osTree.query("Event=='Initiation of Oxidative Stress'")["Function"].unique())
mos_function = list(osTree.query("Event=='Modulation of Oxidative Stress'")["Function"].unique())
oos_function = list(osTree.query("Event=='Outcomes of Oxidative Stress'")["Function"].unique())

In [188]:
ios_function,mos_function,oos_function

(['Reactive Oxygen Species (ROS)',
  'Reactive Nitrogen Species (RNS)',
  'Reactive Aldehydes'],
 ['Redox Metabolites',
  'Antioxidant Enzymes',
  'Redox Regulating Proteins',
  'Antioxidants'],
 ['Protein Oxidation (O-PTMs)',
  'Lipid Peroxidation Products',
  'Oxidative DNA Damage',
  'Nitrative DNA Damage'])

In [189]:
functions =  list(osTree["Function"].unique())
label = label + functions
labelindex = [label.index(x) for x in label]
label2index = {}
for item, index in zip(label,labelindex):
    label2index.update({item:index})

In [190]:
event = ['IOS','MOS','OOS']
for osevent in event:
    if osevent == "IOS":
        for iosfunc in ios_function:
            source.append(label2index[osevent])
            target.append(label2index[iosfunc])
            value.append(scorer(link,"osEvent", "osFunction", 'Initiation of Oxidative Stress', iosfunc))
    if osevent == "MOS":
        for mosfunc in mos_function:
            source.append(label2index[osevent])
            target.append(label2index[mosfunc])
            value.append(scorer(link,"osEvent", "osFunction", 'Modulation of Oxidative Stress', mosfunc))
    if osevent == "OOS":
        for oosfunc in oos_function:
            source.append(label2index[osevent])
            target.append(label2index[oosfunc])
            value.append(scorer(link,"osEvent", "osFunction", 'Outcomes of Oxidative Stress', oosfunc))

#### Functions to MeSH

In [191]:
mesh = list(osTree["Name"].unique())
label = label + mesh
labelindex = [label.index(x) for x in label]
label2index = {}
for item, index in zip(label,labelindex):
    label2index.update({item:index})

In [192]:
for f,m in zip(osTree['Function'], osTree['Name']):
    source.append(label2index[f])
    target.append(label2index[m])
    value.append(scorer(link,"osFunction", "OS", f, m))

### Plotly Shanky

In [194]:
import plotly.graph_objects as go
plt.figure(figsize = [50,50])

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = label,
      color = "blue"
    ),
    link = dict(
      source = source,
      target = target,
      value =  value,
  ))])

fig.update_layout(title_text="OS-CVD Link Prediction", font_size=10, width=1500, 
      height=2500)
fig.write_html("plots/link-os-cvd-mesh.html")
fig.show()

<Figure size 3600x3600 with 0 Axes>